## More on collections

### The definition of the `list` type

So far we've seen the `list` collection type, which is essentially a simply linked list. The definition of `list` is similar to the one we proposed with `MyList`, but
- is general for any type of data
- use proper symbols instead of labels

```fsharp
type List<'T> = 
       | ([])  
       | ( :: )  of Head: 'T * Tail: 'T list
```

> Above you can see the definition of a _generic type_, much in the same way as C++ for example. Also, note that the symbols `[]` and `::` are used as cases in the discriminated union type.

Since a list is a discriminated union, one can use it as that. For example, you can write [a function to print the elements of a list](https://fsharpforfunandprofit.com/posts/match-expression/) matching against the two cases of a `list` type:

In [1]:
// loop through a list and print the values
let rec loopAndPrint aList =
    match aList with
    // empty list means we're done.
    | [] ->
        printfn "empty"

    // binding to head::tail.
    | x::xs ->
        printfn "element=%A," x
        // do all over again with the
        // rest of the list
        loopAndPrint xs

loopAndPrint []        

empty


In [2]:
let l = [1;2;3]

l |> loopAndPrint


element=1,
element=2,
element=3,
empty


Different elements of the list can be accessed through indices, although their use is not widespread, because, as we said before, the lists are used as a whole through the `List.` methods or properly defined recursive functions in the case are needed. In some cases, the `Head` and `Tail` functions are used to find the first element and the rest of the list:

In [3]:
printfn "%A" l.Head
printfn "%A" (l |> List.head)
printfn "%A" l.Tail 
printfn "%A" (l |> List.tail) 

1
1
[2; 3]
[2; 3]


Here we can see two variants of these functions:
- Those of type `List.` that come from the F# list module,
- _Methods_ of a `list` data type.

> In the case of methods, we see how F# also provides features associated with object-oriented programming.

You have to pay attention that a list can be empty, because in that case these methods return an exception:

In [4]:
printfn "%A" [].Head 

Error: System.InvalidOperationException: The input list was empty.
   at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4086
   at <StartupCode$FSI_0009>.$FSI_0009.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

In [5]:
printfn "%A" [].Tail

Error: System.InvalidOperationException: The input list was empty.
   at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4091
   at <StartupCode$FSI_0010>.$FSI_0010.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Using list notation, they can be created like this:

In [6]:
let l = 3 :: [6;7]
printfn "%A" l

[3; 6; 7]


and can be concatenated through the `@` operator:

In [7]:
let l2 = l @ [8;9]
printfn "%A" l2

[3; 6; 7; 8; 9]


### Processing lists

As we have been insisting, the idea of ​​collections is to be able to process them through a function, or a concatenated series of functions (using _piping_) in such a way as to obtain the appropriate result.

We saw that functions on collections can be classified according to the type of inputs and outputs of each of them. Detailing this idea, the following image shows some examples of them, and what types of inputs and outputs they have (from the book [Stylish F#, de Kit Easton](https://link.springer.com/book/10.1007/978-1-4842-7205-3)):

<img src="../img/KEaston-Table4-1.png" alt="" width="400"/>


Most of these functions receive as argument a function and a list, and return a new list with the corresponding transformation. The function is usually written as an anonymous function:

```fsharp
fun arguments -> expression
```

also known as _lambda_ functions.

Let's work with an example to incorporate this concept. To do this, we'll use a pair of `Position` and `Player` data types that describe some characteristics of a soccer player:

In [8]:
type Position = GoalKeeper | Defender | Midfielder | Forward 

type Player =
    {
        Number: int 
        Name : string
        Team: string 
        Position: Position 
        Age: uint8 
    }

Obviously, the example will use the following ⭐️⭐️⭐️ :

In [9]:
let champions2022 = [
    // Los 3 arqueros de Argentina en Qatar 2022
    {Number = 23; Name = "Emiliano Martínez" ;  Team = "Aston Villa"; Position = GoalKeeper; Age = 30uy};
    {Number = 12; Name = "Gerónimo Rulli" ;  Team = "Villarreal"; Position = GoalKeeper; Age = 30uy};
    {Number = 1; Name = "Franco Armani" ;  Team = "River"; Position = GoalKeeper; Age = 36uy};
    // Los 9 defensores de Argentina en Qatar 2022
    {Number = 26; Name = "Nahuel Molina" ;  Team = "Atlético de Madrid"; Position = Defender; Age = 24uy};
    {Number = 4; Name = "Gonzalo Montiel" ;  Team = "Sevilla"; Position = Defender; Age = 25uy};
    {Number = 13; Name = "Cristian Romero" ;  Team = "Tottenham"; Position = Defender; Age = 24uy};
    {Number = 6; Name = "Germán Pezzella" ;  Team = "Betis"; Position = Defender; Age = 31uy};
    {Number = 19; Name = "Nicolás Otamendi" ;  Team = "Benfica"; Position = Defender; Age = 34uy};
    {Number = 25; Name = "Lisandro Martínez" ;  Team = "Manchester United"; Position = Defender; Age = 24uy};
    {Number = 8; Name = "Marcos Acuña" ;  Team = "Sevilla"; Position = Defender; Age = 31uy};
    {Number = 3; Name = "Nicolás Tagliafico" ;  Team = "Olympique de Lyon"; Position = Defender; Age = 30uy};
    {Number = 2; Name = "Juan Foyth" ;  Team = "Villarreal"; Position = Defender; Age = 24uy};
    // Los 8 mediocampistas de Argentina en Qatar 2022
    {Number = 7; Name = "Rodrigo De Paul" ;  Team = "Atlético de Madrid"; Position = Midfielder; Age = 28uy};
    {Number = 5; Name = "Leandro Paredes" ;  Team = "Juventus"; Position = Midfielder; Age = 28uy};
    {Number = 20; Name = "Alexis Mac Allister" ;  Team = "Brighton"; Position = Midfielder; Age = 23uy};
    {Number = 18; Name = "Guido Rodríguez" ;  Team = "Betis"; Position = Midfielder; Age = 28uy};
    {Number = 17; Name = "Alejandro Gómez" ;  Team = "Sevilla"; Position = Midfielder; Age = 34uy};
    {Number = 24; Name = "Enzo Fernández" ;  Team = "Benfica"; Position = Midfielder; Age = 21uy};
    {Number = 14; Name = "Exequiel Palacios" ;  Team = "Bayer Leverkusen"; Position = Midfielder; Age = 24uy};
    {Number = 16; Name = "Thiago Almada" ;  Team = "Atlanta United"; Position = Midfielder; Age = 21uy};
    // Los 6 delanteros de Argentina en Qatar 2022
    {Number = 11; Name = "Ángel Di María" ;  Team = "Juventus"; Position = Forward; Age = 34uy};
    {Number = 22; Name = "Lautaro Martínez" ;  Team = "Inter"; Position = Forward; Age = 25uy};
    {Number = 9; Name = "Julián Álvarez" ;  Team = "Manchester City"; Position = Forward; Age = 22uy};
    {Number = 21; Name = "Paulo Dybala" ;  Team = "Roma"; Position = Forward; Age = 29uy};
    {Number = 15; Name = "Ángel Correa" ;  Team = "Atlético Madrid"; Position = Forward; Age = 27uy};
    {Number = 10; Name = "Lionel Messi" ;  Team = "París Saint-Germain"; Position = Forward; Age = 35uy};
]

Let's define a function that allows us to loop through the list and print it. For that we use `List.iter` which, precisely, iterates the list and returns `unit`:

In [10]:
let almostPrettyPrintList l = 
    l 
    |> List.iter (fun elem -> printfn "%A" elem)

In [11]:
champions2022 
|> almostPrettyPrintList

{ Number = 23
  Name = "Emiliano Martínez"
  Team = "Aston Villa"
  Position = GoalKeeper
  Age = 30uy }
{ Number = 12
  Name = "Gerónimo Rulli"
  Team = "Villarreal"
  Position = GoalKeeper
  Age = 30uy }
{ Number = 1
  Name = "Franco Armani"
  Team = "River"
  Position = GoalKeeper
  Age = 36uy }
{ Number = 26
  Name = "Nahuel Molina"
  Team = "Atlético de Madrid"
  Position = Defender
  Age = 24uy }
{ Number = 4
  Name = "Gonzalo Montiel"
  Team = "Sevilla"
  Position = Defender
  Age = 25uy }
{ Number = 13
  Name = "Cristian Romero"
  Team = "Tottenham"
  Position = Defender
  Age = 24uy }
{ Number = 6
  Name = "Germán Pezzella"
  Team = "Betis"
  Position = Defender
  Age = 31uy }
{ Number = 19
  Name = "Nicolás Otamendi"
  Team = "Benfica"
  Position = Defender
  Age = 34uy }
{ Number = 25
  Name = "Lisandro Martínez"
  Team = "Manchester United"
  Position = Defender
  Age = 24uy }
{ Number = 8
  Name = "Marcos Acuña"
  Team = "Sevilla"
  Position = Defender
  Age = 31uy }
{ Num

#### A detour to interpolated strings

It is not very compact... We can create a function that allows us to print the data of each player in a more elegant way. We are going to use _interpolated strings_ to do this. An _interpolated string_ is a string of characters beginning with the symbol `$`, and enclosing the text in double quotes. Inside it you can use the values ​​between braces `{}`.

In [12]:
let saludo = "Hola"

printfn "{saludo} Mundo"
printfn $"{saludo} Mundo"

{saludo} Mundo
Hola Mundo


In [13]:
let toStringPlayer player = 
    $"{player.Number}: {player.Name} ({player.Age}), {player.Position}, juega en {player.Team}"


let prettyPrintList l = 
    l
    |> List.iter (fun p -> printfn "%s" (toStringPlayer p))  

prettyPrintList champions2022       

23: Emiliano Martínez (30), GoalKeeper, juega en Aston Villa
12: Gerónimo Rulli (30), GoalKeeper, juega en Villarreal
1: Franco Armani (36), GoalKeeper, juega en River
26: Nahuel Molina (24), Defender, juega en Atlético de Madrid
4: Gonzalo Montiel (25), Defender, juega en Sevilla
13: Cristian Romero (24), Defender, juega en Tottenham
6: Germán Pezzella (31), Defender, juega en Betis
19: Nicolás Otamendi (34), Defender, juega en Benfica
25: Lisandro Martínez (24), Defender, juega en Manchester United
8: Marcos Acuña (31), Defender, juega en Sevilla
3: Nicolás Tagliafico (30), Defender, juega en Olympique de Lyon
2: Juan Foyth (24), Defender, juega en Villarreal
7: Rodrigo De Paul (28), Midfielder, juega en Atlético de Madrid
5: Leandro Paredes (28), Midfielder, juega en Juventus
20: Alexis Mac Allister (23), Midfielder, juega en Brighton
18: Guido Rodríguez (28), Midfielder, juega en Betis
17: Alejandro Gómez (34), Midfielder, juega en Sevilla
24: Enzo Fernández (21), Midfielder, juega

The list can be sorted:

In [14]:
champions2022 
|> List.sort 
|> prettyPrintList

1: Franco Armani (36), GoalKeeper, juega en River
2: Juan Foyth (24), Defender, juega en Villarreal
3: Nicolás Tagliafico (30), Defender, juega en Olympique de Lyon
4: Gonzalo Montiel (25), Defender, juega en Sevilla
5: Leandro Paredes (28), Midfielder, juega en Juventus
6: Germán Pezzella (31), Defender, juega en Betis
7: Rodrigo De Paul (28), Midfielder, juega en Atlético de Madrid
8: Marcos Acuña (31), Defender, juega en Sevilla
9: Julián Álvarez (22), Forward, juega en Manchester City
10: Lionel Messi (35), Forward, juega en París Saint-Germain
11: Ángel Di María (34), Forward, juega en Juventus
12: Gerónimo Rulli (30), GoalKeeper, juega en Villarreal
13: Cristian Romero (24), Defender, juega en Tottenham
14: Exequiel Palacios (24), Midfielder, juega en Bayer Leverkusen
15: Ángel Correa (27), Forward, juega en Atlético Madrid
16: Thiago Almada (21), Midfielder, juega en Atlanta United
17: Alejandro Gómez (34), Midfielder, juega en Sevilla
18: Guido Rodríguez (28), Midfielder, juega

We see that the list appears ordered first by number. In this case, `List.sort` uses a generic matcher to find out which item comes before another. However, the list can be sorted according to some criteria using `List.sortBy`:

In [15]:
champions2022 
|> List.sortBy (fun p -> p.Age)
|> prettyPrintList

24: Enzo Fernández (21), Midfielder, juega en Benfica
16: Thiago Almada (21), Midfielder, juega en Atlanta United
9: Julián Álvarez (22), Forward, juega en Manchester City
20: Alexis Mac Allister (23), Midfielder, juega en Brighton
26: Nahuel Molina (24), Defender, juega en Atlético de Madrid
13: Cristian Romero (24), Defender, juega en Tottenham
25: Lisandro Martínez (24), Defender, juega en Manchester United
2: Juan Foyth (24), Defender, juega en Villarreal
14: Exequiel Palacios (24), Midfielder, juega en Bayer Leverkusen
4: Gonzalo Montiel (25), Defender, juega en Sevilla
22: Lautaro Martínez (25), Forward, juega en Inter
15: Ángel Correa (27), Forward, juega en Atlético Madrid
7: Rodrigo De Paul (28), Midfielder, juega en Atlético de Madrid
5: Leandro Paredes (28), Midfielder, juega en Juventus
18: Guido Rodríguez (28), Midfielder, juega en Betis
21: Paulo Dybala (29), Forward, juega en Roma
23: Emiliano Martínez (30), GoalKeeper, juega en Aston Villa
12: Gerónimo Rulli (30), GoalK

Here the anonymous function `fun p -> p.Age` is of type `Player -> int`, and grabs the age of each player.

Perhaps one does not need to print all the data for each player:

In [16]:
champions2022 
|> List.sortBy (fun p -> p.Age)
|> List.map (fun p -> $"{p.Name} tiene {p.Age} años")
|> List.iter (fun s -> printfn "%s" s)

Enzo Fernández tiene 21 años
Thiago Almada tiene 21 años
Julián Álvarez tiene 22 años
Alexis Mac Allister tiene 23 años
Nahuel Molina tiene 24 años
Cristian Romero tiene 24 años
Lisandro Martínez tiene 24 años
Juan Foyth tiene 24 años
Exequiel Palacios tiene 24 años
Gonzalo Montiel tiene 25 años
Lautaro Martínez tiene 25 años
Ángel Correa tiene 27 años
Rodrigo De Paul tiene 28 años
Leandro Paredes tiene 28 años
Guido Rodríguez tiene 28 años
Paulo Dybala tiene 29 años
Emiliano Martínez tiene 30 años
Gerónimo Rulli tiene 30 años
Nicolás Tagliafico tiene 30 años
Germán Pezzella tiene 31 años
Marcos Acuña tiene 31 años
Nicolás Otamendi tiene 34 años
Alejandro Gómez tiene 34 años
Ángel Di María tiene 34 años
Lionel Messi tiene 35 años
Franco Armani tiene 36 años


You can search for an element of the list with a particular characteristic:

In [17]:
champions2022
|> List.find (fun p -> p.Age = 21uy)


{ Number = 24\n Name = "Enzo Fernández"\n Team = "Benfica"\n Position = Midfielder\n Age = 21uy } Number 24 Name Enzo Fernández Team Benfica Position Midfielder Age 21

In [18]:
champions2022
|> List.sortBy (fun p -> p.Number)
|> List.find (fun p -> p.Age = 21uy)

{ Number = 16\n Name = "Thiago Almada"\n Team = "Atlanta United"\n Position = Midfielder\n Age = 21uy } Number 16 Name Thiago Almada Team Atlanta United Position Midfielder Age 21

In [19]:
champions2022
|> List.find (fun p -> p.Age = 18uy)

Error: System.Collections.Generic.KeyNotFoundException: An index satisfying the predicate was not found in the collection.
   at Microsoft.FSharp.Collections.ListModule.Find[T](FSharpFunc`2 predicate, FSharpList`1 list) in D:\a\_work\1\s\src\FSharp.Core\list.fs:line 480
   at <StartupCode$FSI_0024>.$FSI_0024.main@()
   at System.RuntimeMethodHandle.InvokeMethod(Object target, Void** arguments, Signature sig, Boolean isConstructor)
   at System.Reflection.MethodInvoker.Invoke(Object obj, IntPtr* args, BindingFlags invokeAttr)

Note that `List.find` returns the first element it finds, but gives an exception if no element with the required feature is found.

If one wants all elements that share a certain feature, we use `List.filter`

In [20]:
champions2022
|> List.filter (fun p -> p.Age = 21uy)

index value 0 { Number = 24\n Name = "Enzo Fernández"\n Team = "Benfica"\n Position = Midfielder\n Age = 21uy } Number 24 Name Enzo Fernández Team Benfica Position Midfielder Age 21 1 { Number = 16\n Name = "Thiago Almada"\n Team = "Atlanta United"\n Position = Midfielder\n Age = 21uy } Number 16 Name Thiago Almada Team Atlanta United Position Midfielder Age 21

If the predicate condition is not met, the `List.filter` returns an empty list, which is nice:

In [38]:
let championsOfAge18 = 
    champions2022
    |> List.filter (fun p -> p.Age = 18uy)

printfn "%A" championsOfAge18    

[]


In [22]:
champions2022
|> List.filter (fun p -> p.Age > 25uy && p.Position = Defender)

index value 0 { Number = 6\n Name = "Germán Pezzella"\n Team = "Betis"\n Position = Defender\n Age = 31uy } Number 6 Name Germán Pezzella Team Betis Position Defender Age 31 1 { Number = 19\n Name = "Nicolás Otamendi"\n Team = "Benfica"\n Position = Defender\n Age = 34uy } Number 19 Name Nicolás Otamendi Team Benfica Position Defender Age 34 2 { Number = 8\n Name = "Marcos Acuña"\n Team = "Sevilla"\n Position = Defender\n Age = 31uy } Number 8 Name Marcos Acuña Team Sevilla Position Defender Age 31 3 { Number = 3\n Name = "Nicolás Tagliafico"\n Team = "Olympique de Lyon"\n Position = Defender\n Age = 30uy } Number 3 Name Nicolás Tagliafico Team Olympique de Lyon Position Defender Age 30

In [23]:
let arqueros = 
    champions2022
    |> List.filter (fun p -> p.Position = GoalKeeper)

prettyPrintList arqueros    

23: Emiliano Martínez (30), GoalKeeper, juega en Aston Villa
12: Gerónimo Rulli (30), GoalKeeper, juega en Villarreal
1: Franco Armani (36), GoalKeeper, juega en River


We may use other methods to process the listing data:

In [24]:
champions2022
|> List.averageBy (fun p -> p.Age)

Stopped due to error


Error: input.fsx (2,29)-(2,34) typecheck error The type 'uint8' does not support the operator 'DivideByInt'

Interesting, you have to convert the data to `float`:

In [39]:
champions2022
|> List.averageBy (fun p -> float p.Age)

27.76923076923077

There are functions that allow you to process the list, collecting data that has similar characteristics.

In [26]:
champions2022
|> List.groupBy (fun p -> p.Team)

index value 0 (Aston Villa, [{ Number = 23\n Name = "Emiliano Martínez"\n Team = "Aston Villa"\n Position = GoalKeeper\n Age = 30uy }]) Item1 Aston Villa Item2 index value 0 { Number = 23\n Name = "Emiliano Martínez"\n Team = "Aston Villa"\n Position = GoalKeeper\n Age = 30uy } Number 23 Name Emiliano Martínez Team Aston Villa Position GoalKeeper Age 30 1 (Villarreal, [{ Number = 12\n Name = "Gerónimo Rulli"\n Team = "Villarreal"\n Position = GoalKeeper\n Age = 30uy }; { Number = 2\n Name = "Juan Foyth"\n Team = "Villarreal"\n Position = Defender\n Age = 24uy }]) Item1 Villarreal Item2 index value 0 { Number = 12\n Name = "Gerónimo Rulli"\n Team = "Villarreal"\n Position = GoalKeeper\n Age = 30uy } Number 12 Name Gerónimo Rulli Team Villarreal Position GoalKeeper Age 30 1 { Number = 2\n Name = "Juan Foyth"\n Team = "Villarreal"\n Position = Defender\n Age = 24uy } Number 2 Name Juan Foyth Team Villarreal Position Defender Age 24 2 (River, [{ Number = 1\n Name = "Franco Armani"\n Team = "River"\n Position = GoalKeeper\n Age = 36uy }]) Item1 River Item2 index value 0 { Number = 1\n Name = "Franco Armani"\n Team = "River"\n Position = GoalKeeper\n Age = 36uy } Number 1 Name Franco Armani Team River Position GoalKeeper Age 36 3 (Atlético de Madrid, [{ Number = 26\n Name = "Nahuel Molina"\n Team = "Atlético de Madrid"\n Position = Defender\n Age = 24uy }; { Number = 7\n Name = "Rodrigo De Paul"\n Team = "Atlético de Madrid"\n Position = Midfielder\n Age = 28uy }]) Item1 Atlético de Madrid Item2 index value 0 { Number = 26\n Name = "Nahuel Molina"\n Team = "Atlético de Madrid"\n Position = Defender\n Age = 24uy } Number 26 Name Nahuel Molina Team Atlético de Madrid Position Defender Age 24 1 { Number = 7\n Name = "Rodrigo De Paul"\n Team = "Atlético de Madrid"\n Position = Midfielder\n Age = 28uy } Number 7 Name Rodrigo De Paul Team Atlético de Madrid Position Midfielder Age 28 4 (Sevilla, [{ Number = 4\n Name = "Gonzalo Montiel"\n Team = "Sevilla"\n Position = Defender\n Age = 25uy }; { Number = 8\n Name = "Marcos Acuña"\n Team = "Sevilla"\n Position = Defender\n Age = 31uy }; { Number = 17\n Name = "Alejandro Gómez"\n Team = "Sevilla"\n Position = Midfielder\n ... Item1 Sevilla Item2 index value 0 { Number = 4\n Name = "Gonzalo Montiel"\n Team = "Sevilla"\n Position = Defender\n Age = 25uy } Number 4 Name Gonzalo Montiel Team Sevilla Position Defender Age 25 1 { Number = 8\n Name = "Marcos Acuña"\n Team = "Sevilla"\n Position = Defender\n Age = 31uy } Number 8 Name Marcos Acuña Team Sevilla Position Defender Age 31 2 { Number = 17\n Name = "Alejandro Gómez"\n Team = "Sevilla"\n Position = Midfielder\n Age = 34uy } Number 17 Name Alejandro Gómez Team Sevilla Position Midfielder Age 34 5 (Tottenham, [{ Number = 13\n Name = "Cristian Romero"\n Team = "Tottenham"\n Position = Defender\n Age = 24uy }]) Item1 Tottenham Item2 index value 0 { Number = 13\n Name = "Cristian Romero"\n Team = "Tottenham"\n Position = Defender\n Age = 24uy } Number 13 Name Cristian Romero Team Tottenham Position Defender Age 24 6 (Betis, [{ Number = 6\n Name = "Germán Pezzella"\n Team = "Betis"\n Position = Defender\n Age = 31uy }; { Number = 18\n Name = "Guido Rodríguez"\n Team = "Betis"\n Position = Midfielder\n Age = 28uy }]) Item1 Betis Item2 index value 0 { Number = 6\n Name = "Germán Pezzella"\n Team = "Betis"\n Position = Defender\n Age = 31uy } Number 6 Name Germán Pezzella Team Betis Position Defender Age 31 1 { Number = 18\n Name = "Guido Rodríguez"\n Team = "Betis"\n Position = Midfielder\n Age = 28uy } Number 18 Name Guido Rodríguez Team Betis Position Midfielder Age 28 7 (Benfica, [{ Number = 19\n Name = "Nicolás Otamendi"\n Team = "Benfica"\n Position = Defender\n Age = 34uy }; { Number = 24\n Name = "Enzo Fernández"\n Team = "Benfica"\n Position = Midfielder\n Age = 21uy }]) Item1 Benfica Item2 index value 0 { Number = 19\n Name = "Nicolás Otamendi"\n Team = "Benfica"\n Position = Defender\n Age = 34uy } Number 19 Name Nicolás Otamendi Team Ben

The `List.groupBy` function does not return a list of players, but a list of ordered pairs (tuples), where the first element would be the common key of the collected data type, and the second element is a list of elements corresponding to that key:

In [27]:
champions2022
|> List.groupBy (fun p -> p.Team)
|> List.iter (fun (team,l) -> 
                printfn "%s" team 
                prettyPrintList l
                )

Aston Villa
23: Emiliano Martínez (30), GoalKeeper, juega en Aston Villa
Villarreal
12: Gerónimo Rulli (30), GoalKeeper, juega en Villarreal
2: Juan Foyth (24), Defender, juega en Villarreal
River
1: Franco Armani (36), GoalKeeper, juega en River
Atlético de Madrid
26: Nahuel Molina (24), Defender, juega en Atlético de Madrid
7: Rodrigo De Paul (28), Midfielder, juega en Atlético de Madrid
Sevilla
4: Gonzalo Montiel (25), Defender, juega en Sevilla
8: Marcos Acuña (31), Defender, juega en Sevilla
17: Alejandro Gómez (34), Midfielder, juega en Sevilla
Tottenham
13: Cristian Romero (24), Defender, juega en Tottenham
Betis
6: Germán Pezzella (31), Defender, juega en Betis
18: Guido Rodríguez (28), Midfielder, juega en Betis
Benfica
19: Nicolás Otamendi (34), Defender, juega en Benfica
24: Enzo Fernández (21), Midfielder, juega en Benfica
Manchester United
25: Lisandro Martínez (24), Defender, juega en Manchester United
Olympique de Lyon
3: Nicolás Tagliafico (30), Defender, juega en Olymp

Written in a different way, grabbing the tuple elements explicitly:

In [28]:
champions2022
|> List.groupBy (fun p -> p.Team)
|> List.iter (fun t -> 
                printfn "%s" (fst t)
                prettyPrintList (snd t)
                )

Aston Villa
23: Emiliano Martínez (30), GoalKeeper, juega en Aston Villa
Villarreal
12: Gerónimo Rulli (30), GoalKeeper, juega en Villarreal
2: Juan Foyth (24), Defender, juega en Villarreal
River
1: Franco Armani (36), GoalKeeper, juega en River
Atlético de Madrid
26: Nahuel Molina (24), Defender, juega en Atlético de Madrid
7: Rodrigo De Paul (28), Midfielder, juega en Atlético de Madrid
Sevilla
4: Gonzalo Montiel (25), Defender, juega en Sevilla
8: Marcos Acuña (31), Defender, juega en Sevilla
17: Alejandro Gómez (34), Midfielder, juega en Sevilla
Tottenham
13: Cristian Romero (24), Defender, juega en Tottenham
Betis
6: Germán Pezzella (31), Defender, juega en Betis
18: Guido Rodríguez (28), Midfielder, juega en Betis
Benfica
19: Nicolás Otamendi (34), Defender, juega en Benfica
24: Enzo Fernández (21), Midfielder, juega en Benfica
Manchester United
25: Lisandro Martínez (24), Defender, juega en Manchester United
Olympique de Lyon
3: Nicolás Tagliafico (30), Defender, juega en Olymp

Let's see how many players each team contributed:

In [29]:
champions2022
|> List.groupBy (fun p -> p.Team)
|> List.map (fun (t,l) -> (t, l |> List.length))


index value 0 (Aston Villa, 1) Item1 Aston Villa Item2 1 1 (Villarreal, 2) Item1 Villarreal Item2 2 2 (River, 1) Item1 River Item2 1 3 (Atlético de Madrid, 2) Item1 Atlético de Madrid Item2 2 4 (Sevilla, 3) Item1 Sevilla Item2 3 5 (Tottenham, 1) Item1 Tottenham Item2 1 6 (Betis, 2) Item1 Betis Item2 2 7 (Benfica, 2) Item1 Benfica Item2 2 8 (Manchester United, 1) Item1 Manchester United Item2 1 9 (Olympique de Lyon, 1) Item1 Olympique de Lyon Item2 1 10 (Juventus, 2) Item1 Juventus Item2 2 11 (Brighton, 1) Item1 Brighton Item2 1 12 (Bayer Leverkusen, 1) Item1 Bayer Leverkusen Item2 1 13 (Atlanta United, 1) Item1 Atlanta United Item2 1 14 (Inter, 1) Item1 Inter Item2 1 15 (Manchester City, 1) Item1 Manchester City Item2 1 16 (Roma, 1) Item1 Roma Item2 1 17 (Atlético Madrid, 1) Item1 Atlético Madrid Item2 1 18 (París Saint-Germain, 1) Item1 París Saint-Germain Item2 1

This last procedure can be summarized with `List.countBy`:

In [30]:
champions2022
|> List.countBy (fun p -> p.Team)

index value 0 (Aston Villa, 1) Item1 Aston Villa Item2 1 1 (Villarreal, 2) Item1 Villarreal Item2 2 2 (River, 1) Item1 River Item2 1 3 (Atlético de Madrid, 2) Item1 Atlético de Madrid Item2 2 4 (Sevilla, 3) Item1 Sevilla Item2 3 5 (Tottenham, 1) Item1 Tottenham Item2 1 6 (Betis, 2) Item1 Betis Item2 2 7 (Benfica, 2) Item1 Benfica Item2 2 8 (Manchester United, 1) Item1 Manchester United Item2 1 9 (Olympique de Lyon, 1) Item1 Olympique de Lyon Item2 1 10 (Juventus, 2) Item1 Juventus Item2 2 11 (Brighton, 1) Item1 Brighton Item2 1 12 (Bayer Leverkusen, 1) Item1 Bayer Leverkusen Item2 1 13 (Atlanta United, 1) Item1 Atlanta United Item2 1 14 (Inter, 1) Item1 Inter Item2 1 15 (Manchester City, 1) Item1 Manchester City Item2 1 16 (Roma, 1) Item1 Roma Item2 1 17 (Atlético Madrid, 1) Item1 Atlético Madrid Item2 1 18 (París Saint-Germain, 1) Item1 París Saint-Germain Item2 1

A histogram of ages can be constructed:

In [31]:
champions2022
|> List.groupBy (fun p -> p.Age/5uy * 5uy)


index value 0 (30, [{ Number = 23\n Name = "Emiliano Martínez"\n Team = "Aston Villa"\n Position = GoalKeeper\n Age = 30uy }; { Number = 12\n Name = "Gerónimo Rulli"\n Team = "Villarreal"\n Position = GoalKeeper\n Age = 30uy }; { Number = 6\n Name = "Germán Pezzella"\n Team = "Betis"\n Position = Defen... Item1 30 Item2 index value 0 { Number = 23\n Name = "Emiliano Martínez"\n Team = "Aston Villa"\n Position = GoalKeeper\n Age = 30uy } Number 23 Name Emiliano Martínez Team Aston Villa Position GoalKeeper Age 30 1 { Number = 12\n Name = "Gerónimo Rulli"\n Team = "Villarreal"\n Position = GoalKeeper\n Age = 30uy } Number 12 Name Gerónimo Rulli Team Villarreal Position GoalKeeper Age 30 2 { Number = 6\n Name = "Germán Pezzella"\n Team = "Betis"\n Position = Defender\n Age = 31uy } Number 6 Name Germán Pezzella Team Betis Position Defender Age 31 3 { Number = 19\n Name = "Nicolás Otamendi"\n Team = "Benfica"\n Position = Defender\n Age = 34uy } Number 19 Name Nicolás Otamendi Team Benfica Position Defender Age 34 4 { Number = 8\n Name = "Marcos Acuña"\n Team = "Sevilla"\n Position = Defender\n Age = 31uy } Number 8 Name Marcos Acuña Team Sevilla Position Defender Age 31 5 { Number = 3\n Name = "Nicolás Tagliafico"\n Team = "Olympique de Lyon"\n Position = Defender\n Age = 30uy } Number 3 Name Nicolás Tagliafico Team Olympique de Lyon Position Defender Age 30 6 { Number = 17\n Name = "Alejandro Gómez"\n Team = "Sevilla"\n Position = Midfielder\n Age = 34uy } Number 17 Name Alejandro Gómez Team Sevilla Position Midfielder Age 34 7 { Number = 11\n Name = "Ángel Di María"\n Team = "Juventus"\n Position = Forward\n Age = 34uy } Number 11 Name Ángel Di María Team Juventus Position Forward Age 34 1 (35, [{ Number = 1\n Name = "Franco Armani"\n Team = "River"\n Position = GoalKeeper\n Age = 36uy }; { Number = 10\n Name = "Lionel Messi"\n Team = "París Saint-Germain"\n Position = Forward\n Age = 35uy }]) Item1 35 Item2 index value 0 { Number = 1\n Name = "Franco Armani"\n Team = "River"\n Position = GoalKeeper\n Age = 36uy } Number 1 Name Franco Armani Team River Position GoalKeeper Age 36 1 { Number = 10\n Name = "Lionel Messi"\n Team = "París Saint-Germain"\n Position = Forward\n Age = 35uy } Number 10 Name Lionel Messi Team París Saint-Germain Position Forward Age 35 2 (20, [{ Number = 26\n Name = "Nahuel Molina"\n Team = "Atlético de Madrid"\n Position = Defender\n Age = 24uy }; { Number = 13\n Name = "Cristian Romero"\n Team = "Tottenham"\n Position = Defender\n Age = 24uy }; { Number = 25\n Name = "Lisandro Martínez"\n Team = "Manchester United"\n Po... Item1 20 Item2 index value 0 { Number = 26\n Name = "Nahuel Molina"\n Team = "Atlético de Madrid"\n Position = Defender\n Age = 24uy } Number 26 Name Nahuel Molina Team Atlético de Madrid Position Defender Age 24 1 { Number = 13\n Name = "Cristian Romero"\n Team = "Tottenham"\n Position = Defender\n Age = 24uy } Number 13 Name Cristian Romero Team Tottenham Position Defender Age 24 2 { Number = 25\n Name = "Lisandro Martínez"\n Team = "Manchester United"\n Position = Defender\n Age = 24uy } Number 25 Name Lisandro Martínez Team Manchester United Position Defender Age 24 3 { Number = 2\n Name = "Juan Foyth"\n Team = "Villarreal"\n Position = Defender\n Age = 24uy } Number 2 Name Juan Foyth Team Villarreal Position Defender Age 24 4 { Number = 20\n Name = "Alexis Mac Allister"\n Team = "Brighton"\n Position = Midfielder\n Age = 23uy } Number 20 Name Alexis Mac Allister Team Brighton Position Midfielder Age 23 5 { Number = 24\n Name = "Enzo Fernández"\n Team = "Benfica"\n Position = Midfielder\n Age = 21uy } Number 24 Name Enzo Fernández Team Benfica Position Midfielder Age 21 6 { Number = 14\n Name = "Exequiel Palacios"\n Team = "Bayer Leverkusen"\n Position = Midfielder\n Age = 24uy } Number 14 Name Exequiel Palacios Team Bayer Leverkusen Position Midfielder Age 24 7 { Number = 16\n Name = "Thiago Almada"\n Team = "Atlanta United"\n Position = Midfielder\n Age = 21uy } Number 16 Name 

In [32]:
champions2022
|> List.groupBy (fun p -> p.Age/5uy * 5uy)
|> List.map (fun (t,l) -> (t, l.Length))
|> List.sort 

index value 0 (20, 9) Item1 20 Item2 9 1 (25, 7) Item1 25 Item2 7 2 (30, 8) Item1 30 Item2 8 3 (35, 2) Item1 35 Item2 2

The `List.partition` function allows us to group the elements into two lists with disjoint characteristics:

In [41]:
let youngerThan30, olderOnes = 
    champions2022
    |> List.map (fun p -> (p.Name,p.Age))
    |> List.partition (fun (name,age) -> age < 30uy)

printfn "There are %A players younger than 30 and " (youngerThan30 |> List.length)    
printfn "%A players older  " (olderOnes |> List.length)    


There are 16 players younger than 30 and 
10 players older  


Here we use `List.map` to _project_ the full data type into a smaller tuple of `Name` and `Age`, and then instruct the function `partition` to give us players younger than 30 or older ones. The method `List.partition` returns a tuple of lists, the first one with those elements that fullfill the condition given by the predicate `age < 30uy`, and the second one with a list of those elements that do not satisfy the condition. Since the result is a tuple, we unpack it in two values `youngerThan30` and `olderOnes` with the tuple syntax.

Finally, some functions allow us to evaluate a certain characteristic of a list:

In [35]:
champions2022
|> List.forall (fun p -> p.Age < 30uy)

False

In [42]:
youngerThan30
|> List.forall (fun (_,a) -> a < 30uy)


True

Again, check the page [Choosing between collection functions](https://fsharpforfunandprofit.com/posts/list-module-functions/) to query examples and usage of collections methods.